In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from sklearn import metrics
from sklearn.decomposition import PCA
import itertools
import matplotlib.patches as mpatches
from sklearn.neighbors import radius_neighbors_graph
from sklearn.neighbors import kneighbors_graph
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import SpectralClustering
import matplotlib.patches as mpatches
import umap
import bbknn
import bbknn.matrix
from importlib import reload
from scipy import stats
import scipy
import tabulate
import gget

# locals
import utils as ut
reload(ut)

2023-03-04 13:14:34.981128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 13:14:35.111904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-03-04 13:14:35.111924: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-04 13:14:35.138490: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-04 13:14:35.9490

ModuleNotFoundError: No module named 'utils'

In [ ]:
cardDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"
xyPath =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/combinedEmbedding.csv"
xy = pd.read_csv(xyPath)
cTypes = sorted(list(xy['cellTypes'].unique()))
print(cTypes)
print()

keys = ['ND', 'HFD8', 'HFD14']

rna = {}
labels = {}

for key in keys:
    labelPath = f"{cardDir}{key}_macrophage_clusters.csv"
    lf = pd.read_csv(labelPath)
    
    lf = pd.merge(lf, xy[['x', 'y', 'cellId', 'colors']], 
                  how='left',
                  left_on='cellId',
                  right_on='cellId')
    
    labels[key] = lf
    
    rnaPath = f"{cardDir}{key}_macrophage_rna.csv"
    rf = pd.read_csv(rnaPath)
    rf = rf.T
    new_header = rf.iloc[0] 
    rf = rf[1:] 
    rf.columns = new_header 
    rf.index.names = ['cellId']
    
    rna[key] = rf

print('done')

In [ ]:
data = {} # rna dict
refs = {} # label dict

keys = ['ND', 'HFD8', 'HFD14']

ctypes = [
    'Monocytes', 
    'Mac4', 
    'Mac5', 
]

for key in keys:
    rf = rna[key].copy()
    lf = labels[key]
    
    labelMask = (lf['cellType'].isin(ctypes))
    
    # add key flags to barcodes 
    mf = lf[labelMask].reset_index(drop=True)
    mf['cellId'] = mf['cellId'] + "_" + key
    
    rf.index = rf.index + "_" + key
    
    # filter out non-myleiod cells
    rf = rf[rf.index.isin(mf['cellId'])]
    rf = ut.normalize(rf, 1e6)
    
    data[key] = rf
    refs[key] = mf.drop_duplicates(subset='cellId')
    
    print(key, mf.shape, rf.shape)

    
print('done')

In [ ]:
# break

In [ ]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 14, 5
plt.rcParams['figure.facecolor'] = "w"

fig, axs = plt.subplots(1, 3, sharey=True)
axs = axs.ravel()

keys = ['ND', 'HFD8', 'HFD14']
for i, key in enumerate(keys):
    rf = data[key]
    lf = refs[key]
    
    sns.scatterplot(data=lf.sort_values(by='cellType', ascending=False), 
                    x='x',
                    y='y', 
                    hue='cellType', 
                    ec='k', 
                    s=15, 
                    ax=axs[i])
    if i == 2:
        axs[i].legend(loc='upper right')
    else:
        axs[i].legend().remove()
        
    axs[i].set_xticks([])
    axs[i].set_yticks([])
    axs[i].set_ylabel("UMAP 2")
    axs[i].set_xlabel("UMAP 1")
    axs[i].set_title(key)
    
plt.tight_layout()

In [ ]:
# break

In [ ]:
fPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/LR/LewisLabUSCS/Mouse/Mouse-2020-Baccin-LR-pairs.xlsx"

lr = pd.read_excel(fPath)
print(lr.shape)

lr['ligand'] = lr['Ligand.Mouse'].str.upper()
lr['receptor'] = lr['Receptor.Mouse'].str.upper()
lr.head()

In [ ]:
"""GLOBAL DEG """
allLR = list(set(lr['ligand'].to_list() + lr['receptor'].to_list()))
print(f"{len(allLR)=}")


cf = []
xf = []

for i, key in enumerate(keys):
    
    cf.append(data[key])
    xf.append(refs[key])
    
cf = pd.concat(cf)    
xf = pd.concat(xf)    

print(f"{cf.shape=} {xf.shape=}")

# get expressed LR 
expLR = [x for x in allLR if x in rf.columns]
print(f"{len(expLR)=}")


res = []    
for ctype in ctypes:
    cellIds = xf[xf['cellType'] == ctype]['cellId'].to_list()
    
    mask = cf.index.isin(cellIds)
    ingroup = cf[mask]
    outgroup = cf[~mask]
    
    for gene in expLR:
        score, pval = scipy.stats.ranksums(ingroup[gene], 
                                           outgroup[gene],
                                           alternative='two-sided')
        
        inMean = ingroup[gene].mean()
        outMean = outgroup[gene].mean()
        lfc = np.log2(inMean) - np.log2(outMean)
        
        row = {
            'gene' : gene,
            'cellType' : ctype,
            'score' : score,
            'pval' : pval,
            'log2foldchange' : lfc,
            'groupMean' : inMean,
            'otherMean' : outMean,
        }
        res.append(row)
        
res = pd.DataFrame(res)
print(f"{res.shape=}")
res.head()

In [ ]:
def printGSR(ef, top=None, columns=['path_name', 'p_val', 'overlapping_genes']):
    """A function to print pathway results """
    n = top
    if top is None:
        n = len(ef)

    ef['overlapping_genes'] = ef['overlapping_genes'].apply(lambda x: ", ".join(x))
    printDf = ef[columns].head(n)
    print(tabulate.tabulate(printDf, 
                            headers=columns, 
                            showindex=False,
                            tablefmt='orgtbl'))

In [ ]:
alpha = 0.01
top = 25

sig = res[res['pval'] < alpha]
sig = sig.sort_values(by=['cellType', 'log2foldchange'], ascending=[True, False])
sig['geneName'] = sig['gene'].apply(lambda x: str(x).lower().capitalize())
print(sig.shape)


plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 14, 5
plt.rcParams['figure.facecolor'] = "w"

fig, axs = plt.subplots(1, 3)
axs = axs.ravel()


for i, ctype in enumerate(ctypes):
    
    tf = sig[sig['cellType'] == ctype]
    # tf = pd.concat([tf.head(top), tf.tail(top)])
    
    sns.barplot(data=tf.head(top),
                y='geneName', 
                x='log2foldchange',
                ec='k', 
                color='lightgrey',
                zorder=10,
                ax=axs[i])
    
    axs[i].grid(True, zorder=-1)
    axs[i].set_ylabel('')
    axs[i].set_xlabel('Fold Change (log2)')
    axs[i].set_title(ctype)
    print(tf[['cellType', 'geneName', 'log2foldchange']].head(top))
    print()
    
    genes = tf['gene'].head(top).to_list()
    
    # db = 'WikiPathways_2019_Mouse'
    # db = 'KEGG_2019_Mouse'
    db = 'ontology'
    enrich = gget.enrichr(genes, database=db)
    printGSR(enrich, top=20)
    
    
plt.tight_layout()


# sig.head()



In [ ]:
# res.to_csv("~/temp/globalDEG.csv", index=False)
# print("done")

In [ ]:
# break

In [ ]:
""" Explore week 8 in-depth """
allLR = list(set(lr['ligand'].to_list() + lr['receptor'].to_list()))
print(f"{len(allLR)=}")


keys = ['HFD8']

cf = []
xf = []

for i, key in enumerate(keys):
    
    cf.append(data[key])
    xf.append(refs[key])
    
cf = pd.concat(cf)    
xf = pd.concat(xf)    

print(f"{cf.shape=} {xf.shape=}")

# get expressed LR 
expLR = [x for x in allLR if x in rf.columns]
print(f"{len(expLR)=}")


res = []    
for ctype in ctypes:
    cellIds = xf[xf['cellType'] == ctype]['cellId'].to_list()
    
    mask = cf.index.isin(cellIds)
    ingroup = cf[mask]
    outgroup = cf[~mask]
    
    for gene in expLR:
        score, pval = scipy.stats.ranksums(ingroup[gene], 
                                           outgroup[gene],
                                           alternative='two-sided')
        
        inMean = ingroup[gene].mean()
        outMean = outgroup[gene].mean()
        lfc = np.log2(inMean) - np.log2(outMean)
        
        row = {
            'gene' : gene,
            'cellType' : ctype,
            'score' : score,
            'pval' : pval,
            'log2foldchange' : lfc,
            'groupMean' : inMean,
            'otherMean' : outMean,
        }
        res.append(row)
        
res = pd.DataFrame(res)
print(f"{res.shape=}")
res.head()


In [ ]:
alpha = 0.01
top = 25

sig = res[res['pval'] < alpha]
sig = sig.sort_values(by=['cellType', 'log2foldchange'], ascending=[True, False])
sig['geneName'] = sig['gene'].apply(lambda x: str(x).lower().capitalize())
print(sig.shape)


plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 14, 5
plt.rcParams['figure.facecolor'] = "w"

fig, axs = plt.subplots(1, 3)
axs = axs.ravel()


for i, ctype in enumerate(ctypes):
    
    tf = sig[sig['cellType'] == ctype]
    # tf = pd.concat([tf.head(top), tf.tail(top)])
    
    sns.barplot(data=tf.head(top),
                y='geneName', 
                x='log2foldchange',
                ec='k', 
                color='lightgrey',
                zorder=10,
                ax=axs[i])
    
    axs[i].grid(True, zorder=-1)
    axs[i].set_ylabel('')
    axs[i].set_xlabel('Fold Change (log2)')
    axs[i].set_title(ctype)
    print(tf[['cellType', 'geneName', 'log2foldchange']].head(top))
    print()
    
    genes = tf['gene'].head(top).to_list()
    
    # db = 'WikiPathways_2019_Mouse'
    # db = 'KEGG_2019_Mouse'
    db = 'ontology'
    enrich = gget.enrichr(genes, database=db)
    printGSR(enrich, top=20)
    
    
plt.tight_layout()


# sig.head()



In [ ]:
"""
MONOCYTES AT WEEK 8 VS ND
"""

In [ ]:
""" week 8 spatial data """

cardOutDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"

dfList = []

for f in os.listdir(cardOutDir):
    if 'macrophage' in f:
        fullPath = f"{cardOutDir}{f}"
        key = f.split("_")[0]
        df = pd.read_csv(fullPath)
        df = df.rename(columns={'Unnamed: 0' : 'spotId'})
        df['key'] = key
        dfList.append(df)

df = pd.concat(dfList, ignore_index=True)
print(f"{df.shape=}")
print(df['key'].value_counts())
print()
df.head()

In [ ]:
sptDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

normalize = True

spt = {}

for f in os.listdir(sptDir):
    if "spt" in f and "macrophage" in f:
        print(f)
        fullPath = f"{sptDir}{f}"
        key = f.split("_")[0]
        sdf = pd.read_csv(fullPath)
        sdf = sdf.rename(columns={'Unnamed: 0' : 'gene'})
        sdf = sdf.set_index('gene')
        sdf = sdf.T
        sdf.index = sdf.index.str.replace("-", ".")
        if normalize:
            sdf = ut.normalize(sdf)
        spt[key] = sdf
    
print('done')

In [ ]:
key = 'HFD8'
sdf = spt[key]

# get the expressed LRs
workingList = list(set(sig['gene'].to_list()))
print(f"{len(workingList)=}")

expLR = [x for x in workingList if x in sdf.columns]
print(f"{len(expLR)=}")


cardDf = df[df['key'] == key]
cardDf = cardDf.drop_duplicates()
cardDf = cardDf[cardDf['spotId'].isin(sdf.index)]
sdf = sdf[sdf.index.isin(cardDf['spotId'])]

print(f"{cardDf.shape=}")
print(f"{sdf.shape=}")
assert(np.all(sdf.index == cardDf['spotId'])) # make sure the sort order is the same
print()

res = []

for i, ctype in enumerate(ctypes):
    tf = sig[sig['cellType'] == ctype]
    
    # ligands
    gf = lr[lr['ligand'].isin(tf['gene'])]
    
    for idx, row in gf.iterrows():
        lig = row['ligand']
        rec = row['receptor']
        lfc = tf.loc[tf['gene'] == lig]['log2foldchange'].values[0]
        degpval = tf.loc[tf['gene'] == lig]['pval'].values[0]
        
        if "&" in rec or "&" in lig:
            continue
            
        if not rec in sdf.columns or lig not in sdf.columns:
            continue
        
        lrColoc = sdf[[lig, rec]].sum(axis=1)
        lrColoc = np.sum(lrColoc > 0) / len(sdf)
        lrcorr, lrpval = scipy.stats.pearsonr(sdf[lig], sdf[rec])
        
        corr, pval = scipy.stats.pearsonr(sdf[lig], cardDf[ctype])
                
        nrow = {
            'cellType' : ctype,
            'ligand' : lig,
            'receptor' : rec,
            'type' : 'ligand',
            'log2foldchange' : lfc,
            'deg_pval' : degpval,
            'LRcolocalization' : lrColoc,
            'LRcorr' : lrcorr,
            'LRpval' : lrpval,
            'corr' : corr,
            'pval' : pval,
        }
        res.append(nrow)
        
    # receptors
    gf = lr[lr['receptor'].isin(tf['gene'])]
    
    for idx, row in gf.iterrows():
        lig = row['ligand']
        rec = row['receptor']
        
        if "&" in rec or "&" in lig:
            continue
            
        if not rec in sdf.columns or lig not in sdf.columns:
            continue
        
        
        lfc = tf.loc[tf['gene'] == rec]['log2foldchange'].values[0]
        degpval = tf.loc[tf['gene'] == rec]['pval'].values[0]
        lrColoc = sdf[[lig, rec]].sum(axis=1)
        lrColoc = np.sum(lrColoc > 0) / len(sdf)

        corr, pval = scipy.stats.pearsonr(sdf[rec], cardDf[ctype])

        nrow = {
            'cellType' : ctype,
            'ligand' : lig,
            'receptor' : rec,
            'type' : 'receptor',
            'log2foldchange' : lfc,
            'deg_pval' : degpval,
            'LRcolocalization' : lrColoc,
            'corr' : corr,
            'pval' : pval,
        }
        res.append(nrow)

res = pd.DataFrame(res)
print(f"{res.shape=}")
res.head()

In [ ]:
res.to_csv("~/temp/HFD8_lr_DEG.csv")

In [ ]:
break

In [ ]:
alpha = 0.01
lfc = 0.25

pdf = res.copy()
pdf = pdf[pdf['deg_pval'] < alpha]
pdf = pdf[pdf['log2foldchange'] > lfc]

pdf['LR'] = pdf['ligand'] + "_" + pdf['receptor']
pdf['count'] = pdf.groupby('LR')['type'].transform('nunique')
pdf = pdf[pdf['count'] > 1]
pdf = pdf.sort_values(by=['LR', 'type'])

nf = []

for lrPair in pdf['LR'].unique():
    
    bdf = pdf[pdf['LR'] == lrPair]
    
    row = {
        'ligand' : lrPair.split("_")[0].lower().capitalize(),
        'receptor' : lrPair.split("_")[1].lower().capitalize(),
        'sender' : bdf['cellType'].to_list()[0],
        'reciever' : bdf['cellType'].to_list()[1],
        'coloc' : bdf['LRcolocalization'].to_list()[0],
        'spatialCorr' : bdf['LRcorr'].to_list()[0],
        'ligandCorr' : bdf['corr'].to_list()[0],
        'receptorCorr' : bdf['corr'].to_list()[1],
    }
    
    nf.append(row)
    
nf = pd.DataFrame(nf)

cmap = "turbo"
colorMap = dict(zip(sorted(res['cellType'].unique()), ut.ncolor(5, cmap=cmap)))
nf['sColor'] = nf['sender'].map(colorMap)
nf['rColor'] = nf['reciever'].map(colorMap)
nf = nf.sort_values(by='receptor')


plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 8.5, 8.5
plt.rcParams['figure.facecolor'] = "w"

sns.scatterplot(data=nf, 
                x='ligand',
                y='receptor',
                ec=nf['rColor'].to_list(),
                c=nf['sColor'].to_list(),
                linewidth=2.4,
                s=75,
                zorder=2)

plt.ylabel("Receptor")
plt.xlabel("Ligand")
ax = plt.gca()
ax.grid(True, zorder=0)
ax.tick_params(axis='x', labelrotation=90)

handles = []

for ctype in ctypes:
    handle = mpatches.Patch(color=colorMap[ctype], 
                            label=ctype)
    handles.append(handle)
    
plt.legend(handles=handles, bbox_to_anchor=(1.3, 1.02))
# pdf


In [ ]:
alpha = 0.01
lfc = 0.25

pdf = res.copy()
pdf = pdf[pdf['deg_pval'] < alpha]
pdf = pdf[pdf['log2foldchange'] > lfc]

pdf['LR'] = pdf['ligand'] + "_" + pdf['receptor']
pdf['count'] = pdf.groupby('LR')['type'].transform('nunique')
pdf = pdf[pdf['count'] > 1]
pdf = pdf.sort_values(by=['LR', 'type'])

cmap = "tab10"
colorMap = dict(zip(sorted(res['cellType'].unique()), ['r', 'b', 'green']))

nf = []

for lrPair in pdf['LR'].unique():
    
    bdf = pdf[pdf['LR'] == lrPair]
    
    row = {
        'ligand' : lrPair.split("_")[0].lower().capitalize(),
        'receptor' : lrPair.split("_")[1].lower().capitalize(),
        'sender' : bdf['cellType'].to_list()[0],
        'reciever' : bdf['cellType'].to_list()[1],
        'coloc' : bdf['LRcolocalization'].to_list()[0],
        'spatialCorr' : bdf['LRcorr'].to_list()[0],
        'ligandCorr' : bdf['corr'].to_list()[0],
        'receptorCorr' : bdf['corr'].to_list()[1],
    }
    
    nf.append(row)
    
nf = pd.DataFrame(nf)


plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 6, 12
plt.rcParams['figure.facecolor'] = "w"
fig, axs = plt.subplots(3, 1)
axs = axs.ravel()

nf['sColor'] = nf['sender'].map(colorMap)
nf['rColor'] = nf['reciever'].map(colorMap)

for i, ctype in enumerate(ctypes):
    pdf = nf[nf['sender'] == ctype]
    pdf = pdf.sort_values(by='reciever')
    # print(pdf.head())

    
    sns.scatterplot(data=pdf, 
                    y='ligand',
                    x='receptor',
                    ec='k',
                    c=pdf['rColor'].to_list(),
                    linewidth=0.75,
                    size_norm=(0, 0.1),
                    legend=False,
                    size='coloc',
                    ax=axs[i],
                    zorder=2)
    
    axs[i].grid(True, zorder=0)
    axs[i].tick_params(axis='x', labelrotation=90)
    axs[i].set_title(ctype)
    axs[i].set_ylabel(f"Ligands")
    axs[i].set_xlabel(f"Receptors")
    
plt.tight_layout()


handles = []

for ctype in ctypes:
    handle = mpatches.Patch(color=colorMap[ctype], 
                            label=ctype)
    handles.append(handle)
    
plt.legend(handles=handles, bbox_to_anchor=(1.3, 1.02))


In [ ]:
break

In [ ]:

plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 15.5, 15
plt.rcParams['figure.facecolor'] = "w"

fig, axs = plt.subplots(3, 2)

top = 20

for i, ctype in enumerate(ctypes):
    for j, geneType in enumerate(['ligand', 'receptor']):

        mask = (res['cellType'] == ctype) & (res['type'] == geneType)
        pdf = res[mask]
        pdf = pdf.sort_values(by='corr', ascending=False)
        pdf['ligand'] = pdf['ligand'].apply(lambda x: str(x).lower().capitalize())
        pdf['receptor'] = pdf['receptor'].apply(lambda x: str(x).lower().capitalize())
        
        print(pdf.head())
        
        
        break
        
#         sns.scatterplot(data=pdf.head(top), 
#                 x='receptor',
#                 y='ligand',
#                 size='LRcolocalization',
#                 hue='corr',
#                 palette='inferno',
#                 zorder=10,
#                 # hue_norm=(0, 1),
#                 legend=False,
#                 ec='k',
#                 ax=axs[i, j])
        
#         axs[i, j].set_ylabel(f"Ligands")
#         axs[i, j].set_xlabel(f"Receptors")
#         axs[i, j].tick_params(axis='x', labelrotation=90)
#         axs[i, j ].grid(True, zorder=0)
#         axs[i, j].set_title(f"{ctype} {geneType.capitalize()}s")
        

    break

    
plt.tight_layout()

In [ ]:
?ut.makeColorbar

In [ ]:
ut.makeColorbar(cmap='inferno', 
                 width=0.3,
                 hieght=2.5,
                 title="Spatial Correlation", 
                 orientation='vertical',
                 tickLabels=['Low', 'High'])

In [ ]:
break

In [ ]:
np.all(sdf.index == cardDf['spotId'])

In [ ]:
cardDf.head()

In [ ]:
break

In [ ]:
"""
GLOBAL DEG
"""
res = []

for 

# for gene in expLR:
    
#           for lrp in lrPairs:
#             score, pval = scipy.stats.ranksums(inNet[lrp], 
#                                                outNet[lrp],
#                                                alternative='two-sided')
            
#             lfc = np.log2(inNet[lrp].mean()+1) - np.log2(outNet[lrp].mean()+1)
                   
    